In [ ]:
!pip install qdrant_client groq langchain langchain-community langchain-groq langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models
from tqdm import tqdm

# --------------------------
# CONFIG
# --------------------------

COLLECTION_NAME = "astro_physics_papers"
QDRANT_URL="YOUR_QDRANT_URL_HERE"
QDRANT_KEY="YOUR_QDRANT_API_KEY_HERE"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# --------------------------
# LOAD MODEL
# --------------------------
try:
    device = "cuda"
    model = SentenceTransformer(EMBEDDING_MODEL, device=device)
    print("Using gpu")
except Exception:
    device = "cpu"
    model = SentenceTransformer(EMBEDDING_MODEL)
    print("gpu unavailable → Using CPU")

# --------------------------
# INIT QDRANT
# --------------------------
qdrant = QdrantClient(url=QDRANT_URL, api_key=QDRANT_KEY)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Using gpu


In [ ]:

from pydantic import BaseModel, Field

groq_key = "YOUR_GROQ_API_KEY_HERE"

# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

llm = ChatGroq(
    temperature=0,
    model_name="openai/gpt-oss-120b",#可以去看grok上有哪些模型，免費的都可以試試看，看哪個效果比較好，如果都太爛可以跟老師申請經費用openai的gpt-4.1-mini。這個是要收費的，但老師說1000元以內可以拿學校的統編去刷。
    groq_api_key=groq_key,
)


class QueryResponse(BaseModel):
    can_be_answered: bool = Field(
        ..., description="Whether the question can be answered based on the given pdf page."
    )
    summary: str = Field(
        ..., description="A concise summary answer if it can be answered, otherwise an empty string."
    )

#可以調整系統指令
SYSTEM_PROMPT = """
You are an intelligent astro-physics research analyst.
Given:
- A page of an astro physics research paper
- A user question

Your job is to:
1. Determine if the question can be answered from the given page.
2. If yes, provide a clear and detailed summary.
3. If not, respond with can_be_answered = false and summary = "".
Your generated summary has to be 100% consistent with the provided document, and you must not rely on any additional knowledge
"""


structured_llm = llm.with_structured_output(QueryResponse)



In [ ]:
# query = "Explain Quasar Clustering in simple terms" #可以改成你想問的
# #現在是把整個query拿去embed，但應該只有關鍵字對搜尋是有幫助的，可以使用llm或是其他方法提取關鍵字後再embed


# query_vec = model.encode(query).tolist()

# results = qdrant.query_points(
#     collection_name=COLLECTION_NAME,
#     query=query_vec,
#     limit=10 #會回傳幾筆資料回來，可以改多改少試試看
# )
# #現在是每一筆result都會query，可以不一定要全部都query，例如說得到幾個夠好的答案之後就停止

# answers = []
# for point in results.points:
#     response = structured_llm.invoke(
#         input=[
#             {"role": "system", "content": SYSTEM_PROMPT},
#             {"role": "user", "content": f"content of page of the paper:\n{point.payload['text']}\n\nQuestion: {query}"}
#         ]
#     )

#     if response.can_be_answered:
#         print(f"\n✅ Can be answered in paper {point.payload['document']}, page {point.payload['page']}:")
#         print(f"Summary: {response.summary}")

#         answers.append({
#             "title": point.payload.get("document"),
#             "page": point.payload.get("page"),
#             "summary": response.summary
#         })
#     else:
#         print(f"\n❌ Cannot be answered in paper {point.payload['document']}, page {point.payload['page']}")

#   #TODO: 寫出一個方法，能夠從answers裡面選出一個最好的答案作為回覆，或是summarize answers裡面的每一項。建議使用llm

#   #建議的evaluate方法：設計很多query（可以自己想，或是用gpt，但必須確定這些query是否可以至少被一篇論文的內容回答。當然你也可以設計一些完全無關的query，這時候他就應該要回答不出來東西才是對的，而不是隨便亂引用一篇然後胡說八道）
#   #可以先標記好每一個query應該用哪篇論文的哪頁來回答，看query_points能不能成功找到正確的論文以及頁數。也可以先自己寫好每一個query的答案，再來看他生的對不對。我們希望他能夠答對越多題越好

In [ ]:
# def extract_keywords(query: str) -> str:
#     prompt = f"""
#     Extract only the essential keywords from this query for semantic search.
#     Output only a short list of keywords separated by commas.

#     Query: "{query}"
#     """
#     # 開始建立給 LLM 的 prompt。用 f-string（帶 {}）是為了把 query 插入。
#     # 這是給 LLM 的完整指令：
#     # 要它抓關鍵字
#     # 回傳逗號分隔
#     # 把真正的 query 放在 "Query: {}" 裡
#     # 使用三引號，避免跳字元
#     response = llm.invoke(prompt)
#     # 呼叫 LLM，傳入剛剛的 prompt。llm.invoke() 依你現有框架會回傳文字。
#     keywords = response.content.strip()
#     # 把 LLM 輸出的前後空白去掉。

#     return keywords
#     # 回傳關鍵字（逗號分隔字串）。

# query = "Explain Quasar Clustering in simple terms"

# keywords = extract_keywords(query)
# # 呼叫你剛寫的函式，從 Query 取得 keyword。
# print("🔎 Extracted Keywords:", keywords)

# query_vec = model.encode(keywords).tolist()
# # 將關鍵字拿去 embedding，而不是整句 query


# results = qdrant.query_points(
#     collection_name=COLLECTION_NAME,
#     query=query_vec,
#     limit=10
#     #會回傳幾筆資料回來，可以改多改少試試看
# )


# answers = []
# MAX_GOOD_ANSWERS = 3  # 如果找到 3 個可回答的就提前停止

# for point in results.points:
#     response = structured_llm.invoke(
#         input=[
#             {"role": "system", "content": SYSTEM_PROMPT},
#             {"role": "user", "content": f"content of page of the paper:\n{point.payload['text']}\n\nQuestion: {query}"}
#         ]
#     )

#     if response.can_be_answered:
#         print(f"\n✅ Can be answered in paper {point.payload['document']}, page {point.payload['page']}:")
#         print(f"Summary: {response.summary}")

#         answers.append({
#             "title": point.payload.get("document"),
#             "page": point.payload.get("page"),
#             "summary": response.summary
#         })

#         # early stopping：找到足夠內容就停止
#         if len(answers) >= MAX_GOOD_ANSWERS:
#             print("\n⏳ Enough good answers found. Stopping early...")
#             break

#     else:
#         print(f"\n❌ Cannot be answered in paper {point.payload['document']}, page {point.payload['page']}")

In [ ]:
import requests
from bs4 import BeautifulSoup

FOLDER_ID = "1RS8kaxjfnsj3WJIotCE9PswN_rqCVX5v"

title_to_link = {}

def get_link(title):
  return title_to_link.get(title)

def fetch_drive():
    url = f"https://drive.google.com/embeddedfolderview?id={FOLDER_ID}#list"
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    entries = soup.find_all("div", class_="flip-entry")

    results = []

    for entry in entries:
        link_tag = entry.find("a", href=True)
        link = link_tag["href"] if link_tag else None
        if not link:
          continue
        title_div = entry.find("div", class_="flip-entry-title")
        if title_div:
            title_to_link[title_div.get_text(strip=True)] = link

fetch_drive()



In [ ]:
# gr.__version__


In [ ]:
import gradio as gr
from IPython.display import display, Markdown
from pydantic import BaseModel, Field
from typing import List


# ============================================================
# ⚠️ Assumes these already exist in your environment:
# llm, model, qdrant, COLLECTION_NAME
# ============================================================


# ==========================================
# 1. 定義提取關鍵字函式 (完全不變)
# ==========================================
def extract_keywords(query: str) -> str:
    prompt = f"""
    Extract only the essential keywords from this query for semantic search.
    Output only a short list of keywords separated by commas.
    Query: "{query}"
    """
    try:
        response = llm.invoke(prompt)
        return response.content.strip()
    except Exception:
        return query


# ============================================================
# 2. ✅ 核心查詢函式（由你原本程式碼封裝而成，邏輯完全一樣）
# ============================================================

class Citation(BaseModel):
    document: str = Field(..., description="PDF filename")
    page: int = Field(..., description="Page number")


class LLMResponse(BaseModel):
    answer: str = Field(..., description="LaTeX formatted answer paragraph")
    citations: List[Citation]

def answer_query_from_pdfs(user_question: str):

    print(f"🔎 Original Question: {user_question}")

    # --- Keyword extraction ---
    search_term = extract_keywords(user_question)
    print(f"🔑 Search Keywords: {search_term}")

    # --- Vector search ---
    query_vec = model.encode(search_term).tolist()
    search_results = qdrant.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vec,
        limit=5
    )

    # --- Context stuffing (UNCHANGED) ---
    context_pieces = []
    for point in search_results.points:
        doc_name = point.payload.get('document', 'Unknown')
        page_num = point.payload.get('page', '?')
        content = point.payload.get('text', '')

        piece = f"--- Source ID: [{doc_name}, Page {page_num}] ---\n{content}\n"
        context_pieces.append(piece)

    full_context = "\n".join(context_pieces)

    # ✅ UPDATED PROMPT FOR STRUCTURED OUTPUT
    final_prompt = rf"""
You are an expert astro-physics researcher.
Answer the user's question based ONLY on the provided contexts.

User Question: {user_question}

*** CRITICAL INSTRUCTION ***
1. Answer in ONE concise paragraph (3–5 sentences).
2. Use LaTeX for ALL math. Wrap all math in $...$.
3. Only cite documents that appear in the contexts.
4. Return structured output.

--- Contexts ---
{full_context}
--- End ---
"""

    try:
        # ✅ LANGCHAIN STRUCTURED OUTPUT
        structured_llm = llm.with_structured_output(LLMResponse)
        result: LLMResponse = structured_llm.invoke(final_prompt)

        return {
            "response": result.answer,
            "citations": result.citations
        }

    except Exception as e:
        return {
            "response": f"LLM Error: {e}",
            "citations": []
        }


# # ============================================================
# # 3. Google Drive 連結函式（保持原樣）
# # ============================================================
# def get_link(filename):
#     return f"https://example.com/{filename}"


# ============================================================
# 4. ✅ Gradio Chatbot Function
# ============================================================
def chatbot(user_message, history, lang_mode):

    result = answer_query_from_pdfs(user_message)

    res = result["response"]
    citations = result["citations"]

    ref_lines = []
    for c in citations:
        pdf = c.document
        page = c.page
        link = get_link(pdf)
        ref_lines.append(f"- 📄 {pdf} (Page {page}) → {link}")

    ref_block = "\n".join(ref_lines) if ref_lines else "- No source available"

    if lang_mode == "English":
        reply = f"{res}\n\n### References\n{ref_block}"
    else:
        reply = f"{res}\n\n### 參考資料\n{ref_block}"

    return reply

# ============================================================
# 5. ✅ Gradio UI
# ============================================================
with gr.Blocks() as demo:
    gr.Markdown("## 📚 Physics Paper Assistant | 物理論文查詢 Chatbot")
    gr.Markdown("Select your preferred answer language below.\n選擇你希望的回覆語言。")

    lang_mode = gr.Radio(
        ["English", "中文"],
        value="English",
        label="Language Mode | 語言模式"
    )

    chatbot_ui = gr.Chatbot(
        label="Chat History | 對話紀錄",
        render_markdown=True,
        bubble_full_width=False,
        latex_delimiters=[{ "left": "$", "right": "$", "display": False }]
    )

    msg = gr.Textbox(label="Ask your question (輸入你的問題按 Enter)")

    def respond(message, chat_history, lang):
        bot_reply = chatbot(message, chat_history, lang)
        chat_history.append((message, bot_reply))
        return "", chat_history

    msg.submit(respond, [msg, chatbot_ui, lang_mode], [msg, chatbot_ui])




# ============================================================
# 6. Launch
# ============================================================
if __name__ == "__main__":
    demo.launch()


/tmp/ipython-input-3030761626.py:228: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(
/tmp/ipython-input-3030761626.py:228: DeprecationWarning: The 'bubble_full_width' parameter will be removed in Gradio 6.0. This parameter no longer has any effect.
  chatbot_ui = gr.Chatbot(
/tmp/ipython-input-3030761626.py:228: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot_ui = gr.Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1ce1c00ca53b6a6c16.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
